## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vanavara,RU,60.3400,102.2797,-16.67,100,86,1.14,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,53.26,62,75,28.77,broken clouds
2,2,Imeni Poliny Osipenko,RU,52.4201,136.4853,-16.33,100,99,1.74,overcast clouds
3,3,Busselton,AU,-33.6500,115.3333,64.09,79,100,12.95,overcast clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,73.69,64,20,11.50,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? -100
What is the maximum temperature you would like for your trip? 120


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
#                                        (city_data_df["Max Temp"] >= min_temp)].dropna(how='any')
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vanavara,RU,60.3400,102.2797,-16.67,100,86,1.14,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,53.26,62,75,28.77,broken clouds
2,2,Imeni Poliny Osipenko,RU,52.4201,136.4853,-16.33,100,99,1.74,overcast clouds
3,3,Busselton,AU,-33.6500,115.3333,64.09,79,100,12.95,overcast clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,73.69,64,20,11.50,few clouds
5,5,Bluff,NZ,-46.6000,168.3333,50.81,74,11,14.27,few clouds
6,6,Ancud,CL,-41.8697,-73.8203,81.23,54,11,5.41,few clouds
7,7,Hilo,US,19.7297,-155.0900,68.25,87,75,5.75,broken clouds
8,8,Hithadhoo,MV,-0.6000,73.0833,81.55,72,100,19.15,overcast clouds
9,9,Hobart,AU,-42.8794,147.3294,61.05,78,87,1.99,overcast clouds


In [5]:
#Count total rows
preferred_cities_df.notnull().sum()

City_ID                681
City                   681
Country                677
Lat                    681
Lng                    681
Max Temp               681
Humidity               681
Cloudiness             681
Wind Speed             681
Current Description    681
dtype: int64

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.replace('', np.nan)

preferred_cities_df = preferred_cities_df.dropna(how='any')

preferred_cities_df.notnull().sum()


City_ID                677
City                   677
Country                677
Lat                    677
Lng                    677
Max Temp               677
Humidity               677
Cloudiness             677
Wind Speed             677
Current Description    677
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vanavara,RU,-16.67,overcast clouds,60.3400,102.2797,
1,Ushuaia,AR,53.26,broken clouds,-54.8000,-68.3000,
2,Imeni Poliny Osipenko,RU,-16.33,overcast clouds,52.4201,136.4853,
3,Busselton,AU,64.09,overcast clouds,-33.6500,115.3333,
4,Port Elizabeth,ZA,73.69,few clouds,-33.9180,25.5701,
5,Bluff,NZ,50.81,few clouds,-46.6000,168.3333,
6,Ancud,CL,81.23,few clouds,-41.8697,-73.8203,
7,Hilo,US,68.25,broken clouds,19.7297,-155.0900,
8,Hithadhoo,MV,81.55,overcast clouds,-0.6000,73.0833,
9,Hobart,AU,61.05,overcast clouds,-42.8794,147.3294,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
#Visualize DF
hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vanavara,RU,-16.67,overcast clouds,60.3400,102.2797,
1,Ushuaia,AR,53.26,broken clouds,-54.8000,-68.3000,Albatross Hotel
2,Imeni Poliny Osipenko,RU,-16.33,overcast clouds,52.4201,136.4853,Yubileynaya
3,Busselton,AU,64.09,overcast clouds,-33.6500,115.3333,Observatory Guest House
4,Port Elizabeth,ZA,73.69,few clouds,-33.9180,25.5701,39 On Nile Guest House
5,Bluff,NZ,50.81,few clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
6,Ancud,CL,81.23,few clouds,-41.8697,-73.8203,Hotel Arena Gruesa
7,Hilo,US,68.25,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Hithadhoo,MV,81.55,overcast clouds,-0.6000,73.0833,Scoop Guest House
9,Hobart,AU,61.05,overcast clouds,-42.8794,147.3294,Mantra on Collins Hobart


In [11]:
#sum rows
hotel_df.notnull().sum()

City                   677
Country                677
Max Temp               677
Current Description    677
Lat                    677
Lng                    677
Hotel Name             677
dtype: int64

In [12]:
#Replace blanks w/ NaN
hotel_df = hotel_df.replace('', np.nan)

In [13]:
#Count Nulls
hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             56
dtype: int64

In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')


In [15]:
#count rows 
clean_hotel_df.notnull().sum()

City                   621
Country                621
Max Temp               621
Current Description    621
Lat                    621
Lng                    621
Hotel Name             621
dtype: int64

In [16]:
#display df
clean_hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ushuaia,AR,53.26,broken clouds,-54.8000,-68.3000,Albatross Hotel
2,Imeni Poliny Osipenko,RU,-16.33,overcast clouds,52.4201,136.4853,Yubileynaya
3,Busselton,AU,64.09,overcast clouds,-33.6500,115.3333,Observatory Guest House
4,Port Elizabeth,ZA,73.69,few clouds,-33.9180,25.5701,39 On Nile Guest House
5,Bluff,NZ,50.81,few clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
6,Ancud,CL,81.23,few clouds,-41.8697,-73.8203,Hotel Arena Gruesa
7,Hilo,US,68.25,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Hithadhoo,MV,81.55,overcast clouds,-0.6000,73.0833,Scoop Guest House
9,Hobart,AU,61.05,overcast clouds,-42.8794,147.3294,Mantra on Collins Hobart
10,Teknaf,BD,66.97,clear sky,20.8624,92.3058,Forestry Guesthouse


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
#Create Figure 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))